# 03 - Merge of ZH Wärmekataster and BFE Heating Demand Model
### Author: Daniel Herrera-Russert
#### January 30, 2025

In [29]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

from owslib.wfs import WebFeatureService
import geopandas as gpd
import requests
from io import BytesIO

import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pyproj import Transformer # for converting coordinates

# Set the default renderer for JupyterLab
pio.renderers.default = 'iframe'

## 1. Loading the Dataset

In this phase of data collection and integration, we merge the **Zürich Wärmekataster** dataset on geothermal probes—already extracted, transformed, and analyzed—with the **heat demand model**, which was loaded as a raster GeoPackage file.

A spatial query has been conducted to isolate geothermal pump locations within the boundaries of the heat demand model. This process ensures that each pump is assigned the corresponding heat demand data from the raster model. The resulting dataset is a relational table where **`gml_id`** serves as the unique key linking the two sources.

Next, we proceed with merging both datasets, addressing the issue of pump locations that fall outside the heat demand coverage, and generating visualizations to explore the data.

In [12]:
# Step 1: Load the borehole dataset and save as GeoPandas
geojson_path = "data/transformed/zh_geothermal_probes.geojson"
zh_geothermal_probes_gdf = gpd.read_file(geojson_path)
zh_geothermal_probes_gdf.head()

,gml_id,x,y,Waermeentnahme,Waermeeintrag,Sondentiefe,Gesamtsondenzahl,GBS-Nummer,Bohrprofil,lon,lat,geometry
0,erdwaermesonden.1464734,2684789,1248345,20.4,0.0,250,3.0,b 00-10370,0,8.561441,47.380481,POINT (2684788.902 1248344.823)
1,erdwaermesonden.1464735,2684962,1248289,68.0,0.0,250,10.0,b 00-10371,1,8.563721,47.379955,POINT (2684961.902 1248288.82)
2,erdwaermesonden.1464736,2684633,1248535,12.5,6.0,250,2.0,b 00-10287,0,8.559411,47.382209,POINT (2684632.902 1248534.826)
3,erdwaermesonden.1464737,2684626,1248526,12.5,6.0,250,2.0,b 00-10288,0,8.559317,47.382129,POINT (2684625.901 1248525.826)
4,erdwaermesonden.1464738,2684623,1248519,12.5,6.0,250,2.0,b 00-10289,0,8.559276,47.382067,POINT (2684622.901 1248518.826)


In [4]:
# Load the transformed BFE heat demand model data
heat_demand_path = "data/transformed/boreholes_with_demand.csv"
heat_demand_df = pd.read_csv(heat_demand_path)
heat_demand_df.head()

,gml_id,OBJECTID,NEEDHOME,NEEDSERVICE,PERCENTGAS,PERCENTOIL,PERCENTPUMP,PERCENTREMOTEHEAT,STYLE
0,erdwaermesonden.1464734,153369.0,719.6,78.5,14.3,57.1,28.6,0.0,3.0
1,erdwaermesonden.1464735,153613.0,383.4,10.5,0.0,14.3,85.7,0.0,2.0
2,erdwaermesonden.1464736,153245.0,565.1,55.5,20.0,40.0,40.0,0.0,3.0
3,erdwaermesonden.1464737,153245.0,565.1,55.5,20.0,40.0,40.0,0.0,3.0
4,erdwaermesonden.1464738,153245.0,565.1,55.5,20.0,40.0,40.0,0.0,3.0


In [7]:
# Merge the two datasets on GBS_ID
merged_df = pd.merge(zh_geothermal_probes_gdf, heat_demand_df, on="gml_id", how="inner")

# Display the first few rows to verify the merge
merged_df.head()

,gml_id,x,y,Waermeentnahme,Waermeeintrag,Sondentiefe,Gesamtsondenzahl,GBS-Nummer,Bohrprofil,lon,lat,geometry,OBJECTID,NEEDHOME,NEEDSERVICE,PERCENTGAS,PERCENTOIL,PERCENTPUMP,PERCENTREMOTEHEAT,STYLE
0,erdwaermesonden.1464734,2684789,1248345,20.4,0.0,250,3.0,b 00-10370,0,8.561441,47.380481,POINT (2684788.902 1248344.823),153369.0,719.6,78.5,14.3,57.1,28.6,0.0,3.0
1,erdwaermesonden.1464735,2684962,1248289,68.0,0.0,250,10.0,b 00-10371,1,8.563721,47.379955,POINT (2684961.902 1248288.82),153613.0,383.4,10.5,0.0,14.3,85.7,0.0,2.0
2,erdwaermesonden.1464736,2684633,1248535,12.5,6.0,250,2.0,b 00-10287,0,8.559411,47.382209,POINT (2684632.902 1248534.826),153245.0,565.1,55.5,20.0,40.0,40.0,0.0,3.0
3,erdwaermesonden.1464737,2684626,1248526,12.5,6.0,250,2.0,b 00-10288,0,8.559317,47.382129,POINT (2684625.901 1248525.826),153245.0,565.1,55.5,20.0,40.0,40.0,0.0,3.0
4,erdwaermesonden.1464738,2684623,1248519,12.5,6.0,250,2.0,b 00-10289,0,8.559276,47.382067,POINT (2684622.901 1248518.826),153245.0,565.1,55.5,20.0,40.0,40.0,0.0,3.0


---

## 2. Data Quality

We quickly produce a missing value assessment, as we already know that a number of probe points are not found within the raster units (pixels) from the energy demand model.

In [27]:
# Count the number of NaN values in each column
nan_counts = merged_df.isna().sum()

# Calculate the percentage of missing values for each column
nan_percentage = (nan_counts / len(merged_df)) * 100

# Create a DataFrame to display the results
nan_summary = pd.DataFrame({"Missing Count": nan_counts, "Percentage": nan_percentage})
nan_summary[12:]

,Missing Count,Percentage
OBJECTID,2896,8.128895
NEEDHOME,2896,8.128895
NEEDSERVICE,2896,8.128895
PERCENTGAS,3013,8.457306
PERCENTOIL,3013,8.457306
PERCENTPUMP,3013,8.457306
PERCENTREMOTEHEAT,3013,8.457306
STYLE,2896,8.128895


The missing values in the **energy demand data** range between **8.12% and 8.46%**. Given the initial dataset size of approximately **35K data points**, this level of missing data is currently acceptable. However, we may consider **imputation strategies** to fill in these gaps. Possible approaches include **mean imputation** or, if time permits, the use of a **machine learning model** to reconstruct the missing values with higher accuracy. The final decision on how to handle these missing values will be made as the project progresses.

In [35]:
# Create a copy of the dataset to avoid modifying the original
plot_data = merged_df.copy()

# Create a new column to classify rows based on whether they contain NaN values
plot_data["Data_Status"] = plot_data.isna().any(axis=1).map({True: "Missing Data (NaN)", False: "Has Data"})

nan_count = plot_data["Data_Status"].value_counts()

fig_bar = px.bar(
    x=nan_count.index,
    y=nan_count.values,
    color=nan_count.index,
    title="Count of Rows with and without Missing Data",
    labels={"x": "Data Status", "y": "Count"},
    color_discrete_map={"Has Data": "teal", "Missing Data (NaN)": "red"}  # Match map colors
)

fig_bar.show()

---

## 3. Visualizations of the Heat Demand Model for Kanton Zürich

A quick geographical plot will serve as a first look at the merged data. For now, the NaN values are being discarded from the visualization, as this is a preliminary assessment of the data.

In [26]:
# Create a copy of the merged dataset to avoid modifying the original data
plot_data = merged_df.copy()

# Drop ALL rows that contain NaN values in ANY column
plot_data = plot_data.dropna()

# Reset index after dropping NaN values
plot_data.reset_index(drop=True, inplace=True)

# Ensure NEEDHOME is numeric (for proper visualization)
plot_data["NEEDHOME"] = pd.to_numeric(plot_data["NEEDHOME"], errors="coerce")

# Apply power transformation for better granularity at high values
plot_data["scaled_NEEDHOME"] = np.power(plot_data["NEEDHOME"], 0.5)  # Square root transformation

# Sort the dataset by residential demand in ascending order (so high values appear on top)
plot_data_sorted = plot_data.sort_values(by="NEEDHOME", ascending=True)

# Define real value ticks for the color bar (adjusted to focus on high values)
tick_values = [0, 100, 500, 1000, 5000, 10000, 25000]  # Adjusted for large values
tick_labels = [str(v) for v in tick_values]  # Convert to strings for display

# Create the scatter map with power-transformed color scale but real values in legend
fig = px.scatter_mapbox(
    plot_data_sorted,  # Use the sorted dataset
    lat="lat",  # Latitude column
    lon="lon",  # Longitude column
    hover_name="gml_id",  # Show gml_id as the main hover label
    hover_data=None,  # Disable automatic hover data
    color="scaled_NEEDHOME",  # Power-transformed for better visibility of high values
    color_continuous_scale="RdPu",  # High contrast for large values
    title="Residential Heat Demand in Kanton Zürich (Power-Adjusted Color)",
    mapbox_style="carto-positron",  # Use OpenStreetMap tiles
    zoom=9,  # Adjust zoom level for your data
    height=800,  # Set height of the map
    size_max=12,  # Maximum size of the points for balance
    color_continuous_midpoint=np.percentile(plot_data_sorted["scaled_NEEDHOME"], 75)  # Focus color on higher values
)

# Update hovertemplate to show original NEEDHOME values (not transformed)
fig.update_traces(
    hovertemplate=(
        "<b>ID: %{hovertext}</b><br>"  # Show gml_id
        "Residential Demand: %{customdata[0]:.1f} MWh/a<extra></extra>"  # Show original value, not transformed
    ),
    customdata=plot_data_sorted[["NEEDHOME"]].values  # Pass original NEEDHOME values for hover
)

# Update the color bar to show real NEEDHOME values instead of transformed ones
fig.update_layout(
    coloraxis_colorbar=dict(
        title="Residential Demand (MWh/a)",  # Show the real value label
        tickvals=np.power(tick_values, 0.5),  # Convert real values to power-transformed scale
        ticktext=tick_labels,  # Show real values as labels
    )
)

# Show the map
fig.show()

In [22]:
# Create a copy of the merged dataset to avoid modifying the original data
plot_data_service = merged_df.copy()

# Drop ALL rows that contain NaN values in ANY column
plot_data_service = plot_data_service.dropna()

# Reset index after dropping NaN values
plot_data_service.reset_index(drop=True, inplace=True)

# Ensure NEEDSERVICE is numeric (for proper visualization)
plot_data_service["NEEDSERVICE"] = pd.to_numeric(plot_data_service["NEEDSERVICE"], errors="coerce")

# Apply log transformation for better visualization of small values (only for color scale)
plot_data_service["log_NEEDSERVICE"] = np.log1p(plot_data_service["NEEDSERVICE"])  # log(x+1) to handle zeros

# Sort the dataset by commercial heat demand in ascending order (so high values appear on top)
plot_data_service_sorted = plot_data_service.sort_values(by="NEEDSERVICE", ascending=True)

# Define real value ticks for the color bar (e.g., 10, 100, 500, 1000, etc.)
tick_values = [0, 10, 50, 100, 500, 1000, 5000]  # Adjust as needed
tick_labels = [str(v) for v in tick_values]  # Convert to strings for display

# Create the scatter map with log-transformed color scale but real values in legend
fig = px.scatter_mapbox(
    plot_data_service_sorted,  # Use the sorted dataset
    lat="lat",  # Latitude column
    lon="lon",  # Longitude column
    hover_name="gml_id",  # Show gml_id as the main hover label
    hover_data=None,  # Disable automatic hover data
    color="log_NEEDSERVICE",  # Log-transformed for better visualization
    color_continuous_scale="YlOrRd",  # High-contrast perceptual color scale
    title="Commercial Heat Demand in Kanton Zürich (Log-Adjusted Color)",
    mapbox_style="carto-positron",  # Use OpenStreetMap tiles
    zoom=9,  # Adjust zoom level for your data
    height=800,  # Set height of the map
    size_max=12,  # Maximum size of the points for balance
)

# Update hovertemplate to show original NEEDSERVICE values (not log-transformed)
fig.update_traces(
    hovertemplate=(
        "<b>ID: %{hovertext}</b><br>"  # Show gml_id
        "Commercial Demand: %{customdata[0]:.1f} MWh/a<extra></extra>"  # Show original value, not log-transformed
    ),
    customdata=plot_data_service_sorted[["NEEDSERVICE"]].values  # Pass original NEEDSERVICE values for hover
)

# Update the color bar to show real NEEDSERVICE values instead of log values
fig.update_layout(
    coloraxis_colorbar=dict(
        title="Commercial Demand (MWh/a)",  # Show the real value label
        tickvals=np.log1p(tick_values),  # Convert real values to log scale
        ticktext=tick_labels,  # Show real values as labels
    )
)

# Show the map
fig.show()